# L4: Optimize DSPy Agent with DSPy Optimizer

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.</p>

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>
</div>

In [ ]:
!pip install python-dotenv==1.0.0 dspy==2.6.23 mlflow==2.21.3 openai==1.61.0 pydantic==2.11.3 --quiet

In [ ]:
!pip install -q langgraph pgvector langchain pypdf sentence-transformers psycopg langchain-community langchain-anthropic tavily-python pandas openai --quiet
# !pip install tavily-python 
!pip install pymilvus==2.3.6 --quiet
!pip install -U tavily-python langchain_community sentence-transformers --quiet
!pip install autogen-agentchat~=0.2 autogen psutil --quiet
# !pip install -q langchain-openai termcolor langchain_community duckduckgo_search wikipedia openapi-python-client==0.12.3 langgraph langchain_experimental yfinance
!pip install -q langchain-openai langchain-anthropic termcolor langchain_community duckduckgo_search wikipedia openapi-python-client langgraph langchain_experimental openai --quiet

In [ ]:
import mlflow

In [ ]:
# from helper import get_mlflow_tracking_uri

mlflow_tracking_uri = "http://mlflow-server.mlflow.svc.cluster.local:8080"
mlflow.set_tracking_uri(mlflow_tracking_uri)

In [ ]:
mlflow.set_experiment("dspy_course_4_")

In [ ]:
mlflow.dspy.autolog(log_evals=True, log_compiles=True, log_traces_from_compile=True)

In [ ]:
import getpass
import os
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

os.environ["TAVILY_API_KEY"] = "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"
os.environ['ATHINA_API_KEY'] = "IhrJrr0krTMRA9ogqi5aaD4ZuYuvMcdG"


INFERENCE_SERVER_URL = "http://localhost:8989"
# MODEL_NAME="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"
MODEL_NAME="ibm-granite/granite-3.3-2b-instruct"
API_KEY= "alanliuxiang"

from openai import OpenAI
client = OpenAI(
    base_url= f"{INFERENCE_SERVER_URL}/v1",
    api_key=API_KEY,
)


In [ ]:
import dspy
# dspy.settings.configure(lm=dspy.LM("openai/gpt-4o-mini"))
# model="huggingface/meta-llama/Llama-3.2-1B"
lm_local = dspy.LM(
    # "ibm-granite/granite-3.3-2b-instruct",
    "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    custom_llm_provider="openai",
    api_base=f"{INFERENCE_SERVER_URL}/v1",  # local endpoint
    api_key=API_KEY,  # empty api_key for local endpoint
    model_type='chat',  # specify chat model type
    cache=False  # disable caching
)

dspy.settings.configure(lm = lm_local)

## Build a RAG Agent

In [ ]:
def search_wikipedia(query: str) -> list[str]:
    results = dspy.ColBERTv2(url="http://20.102.90.50:2017/wiki17_abstracts")(query, k=3)
    return [x["text"] for x in results]

react = dspy.ReAct("question -> answer", tools=[search_wikipedia])

In [ ]:
import json

# Load trainset
trainset = []
with open("trainset.jsonl", "r") as f:
    for line in f:
        trainset.append(dspy.Example(**json.loads(line)).with_inputs("question"))

# Load valset
valset = []
with open("valset.jsonl", "r") as f:
    for line in f:
        valset.append(dspy.Example(**json.loads(line)).with_inputs("question"))

In [ ]:
# Overview of the dataset.
print(trainset[0])

In [ ]:
tp = dspy.MIPROv2(
    metric=dspy.evaluate.answer_exact_match,
    auto="light",
    num_threads=16
)

In [ ]:
dspy.cache.load_memory_cache("./memory_cache.pkl")

In [ ]:
optimized_react = tp.compile(
    react,
    trainset=trainset,
    valset=valset,
    requires_permission_to_run=False,
)

2025/06/14 04:01:35 WARNING mlflow.tracing.export.mlflow: Failed to log trace to MLflow backend. Error: API request to http://mlflow-server.mlflow.svc.cluster.local:8080/api/2.0/mlflow-artifacts/artifacts/36/traces/6d9ec530b9124b6b8e151ea6751c6eb4/artifacts/traces.json failed with exception HTTPConnectionPool(host='mlflow-server.mlflow.svc.cluster.local', port=8080): Max retries exceeded with url: /api/2.0/mlflow-artifacts/artifacts/36/traces/6d9ec530b9124b6b8e151ea6751c6eb4/artifacts/traces.json (Caused by ResponseError('too many 500 error responses')). For full traceback, set logging level to debug.
  4%|▍         | 4/100 [19:36<7:57:42, 298.57s/it]04:01:45 - LiteLLM:ERROR: litellm_logging.py:3482 - Error creating standard logging object - __annotations__
Traceback (most recent call last):
  File "/opt/app-root/lib64/python3.11/site-packages/litellm/litellm_core_utils/litellm_logging.py", line 3464, in get_standard_logging_object_payload
    model_parameters=ModelParamHelper.get_stan

In [ ]:
optimized_react.react.signature

In [ ]:
optimized_react.react.demos

In [ ]:
evaluator = dspy.Evaluate(
    metric=dspy.evaluate.answer_exact_match,
    devset=valset,
    display_table=True,
    display_progress=True,
    num_threads=24,
)

In [ ]:
original_score = evaluator(react)
print(f"Original score: {original_score}")

In [ ]:
optimized_score = evaluator(optimized_react)
print(f"Optimized score: {optimized_score}")